In [2]:
import pandas as pd
from neo4j import GraphDatabase

In [61]:
association_df = pd.read_csv('./service_associations_filled.csv', index_col='Unnamed: 0')
association_df = association_df.drop(columns=['Unnamed: 75'])

In [62]:
association_df.head(3)

,service,GTtXZCegOZ Odoo,7m30TaadKw ownCloud,ddy6eGz0GS GitLab,AuoUzp7zT5 WordPress,ftB5W0yalq PrestaShop,1wjtGuM9P1 Drupal,vfroVvgRms Jenkins,07s0zeZYjN Domain Name,etez96nrAp 4Hx24x7,...,f9WeJggEuN Gibbon,SkmrEaGFJU Seafile,vnFUakntku Bagisto,CigxgBew94 OpenSIS,1faS6yFowN OSPOS,2ox817TNFc Chamilo,YZIeBxQlsV Akaunting,En9HfOMwXA Sign Server,7HZTSufu92 Nexus Repository OSS,GiVGovTMPj OpenEMR
0,GTtXZCegOZ Odoo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7m30TaadKw ownCloud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ddy6eGz0GS GitLab,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

In [20]:
def create_friend_of(tx, name, friend):
    tx.run("MATCH (a:Person) WHERE a.name = $name "
           "CREATE (a)-[:KNOWS]->(:Person {name: $friend})",
           name=name, friend=friend)

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Bob")

with driver.session() as session:
    session.write_transaction(create_friend_of, "Alice", "Carl")


In [21]:
def get_friends_of(tx, name):
    friends = []
    result = tx.run("MATCH (a:Person)-[:KNOWS]->(f) "
                         "WHERE a.name = $name "
                         "RETURN f", name=name)
    for record in result:
        # friends.append(record["friend"])
        friends.append(record)
    return friends

with driver.session() as session:
    friends = session.read_transaction(get_friends_of, "Alice")
    for friend in friends:
        print(friend)

driver.close()

<Record f=<Node id=1 labels=frozenset({'Person'}) properties={'name': 'Bob'}>>
<Record f=<Node id=2 labels=frozenset({'Person'}) properties={'name': 'Carl'}>>


In [64]:
def create_service(tx, service_id, name):
    tx.run("CREATE (s:Service {serviceId: $service_id, name: $name})", service_id=service_id, name=name)

In [65]:
with driver.session() as session:
    for column in association_df.columns:
        if column == 'service':
            continue
        else:
            splits = column.split(' ')
            service_id = splits[0]
            name = ' '.join(splits[1:])
            session.write_transaction(create_service, service_id, name)


In [70]:
def make_asociation_of(tx, service1, service_2):
    tx.run("MATCH (s1:Service {serviceId: $service_id1})"
            "MATCH (s2:Service {serviceId: $service_id2})"
            "CREATE (s1)-[:WORKS_WITH]->(s2)", service_id1=service1, service_id2=service_2)

In [71]:
with driver.session() as session:
    for column1 in association_df.columns:
        if column == 'service':
            continue
        else:
            for column2 in association_df.service.values:
                if association_df[association_df['service'] == column2][column1].values[0] == 1:
                    session.write_transaction(make_asociation_of, column1.split(' ')[0], column2.split(' ')[0])

driver.close()

In [85]:
def get_associated_items(tx, name):
    items = []
    result = tx.run("MATCH (s:Service) -[:WORKS_WITH]- (t:Service {name: $name}) RETURN s", name=name)
    for record in result:
        items.append(record)
    return items

In [87]:
with driver.session() as session:
    items = session.read_transaction(get_associated_items, "WordPress")
    items_names = {item['s']['name'] for item in items}
    print(items_names)

driver.close()

{'phpBB', 'Mautic', 'OSPOS', 'LimeSurvey', 'Domain Name', 'Matomo', 'Live Helper Chat', 'SurveyProject', 'XWiki', 'Akaunting', 'Invoice Ninja', 'ORO CRM', 'MediaWiki', 'DokuWiki'}
